In [1]:
!pip install google-generativeai
!pip install selenium webdriver-manager


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
import google.generativeai as genai
import json
import os
# Set your API key (assumes you have a service account key JSON or API key)
api_key = os.getenv("GEMINI_API_KEY")

if api_key:
    genai.configure(api_key=api_key)
    print("Gemini API key configured successfully from environment variable.")
else:
    print("Error: GEMINI_API_KEY environment variable not found.")
    # You might want to raise an exception or exit the script here
    # raise ValueError("GEMINI_API_KEY environment variable not set.")


Gemini API key configured successfully from environment variable.


In [7]:
def ask_gemini(prompt, model="gemini-2.5-flash", temperature=0):
    """
    Call Gemini 2.5 Flash for a single-turn generative response.
    """
    model = genai.GenerativeModel(model_name=model)
    response = model.generate_content(
        prompt,
        generation_config=genai.types.GenerationConfig(temperature=temperature)
    )
    return response.text


In [45]:
import csv 
import time 

import json

def generate_field_answers(fields, resume_json):
    answers = []
    for field in fields:
        question = field["field_name"]
        options_text = "; ".join(field["options"]) if field["options"] else ""
        
        prompt = f"""
You are a LinkedIn Easy Apply assistant. Your goal is to answer the field in a way that:
1️⃣ Favors the candidate by highlighting their skills, experience, and strengths from the resume.
2️⃣ Favors the recruiter/company by showing alignment with the job and professional fit.

Resume JSON:
{json.dumps(resume_json)}

Field question: {question}
Field type: {field['field_type']}
Current value: {field['value']}
Options (if any): {options_text}

Instructions:
- if resume doesn't have the answer , then provide a professional, concise, and truthful answer. If resume does not provide an explicit answer, infer a suitable response that represents the candidate positively while aligning with company interests.
- also check if field is numerical or date or time or any other format, if so then provide the answer in the same format.
- for salary kind of fields, provide the answer in the format of 100000-200000.
- for experience kind of fields, provide the answer in the format of 2 years . if i have 2 years 3 months, then provide the answer as 2.5 years. if i have 2 years 3 months, then provide the answer as 2 years. if i have 2 years 8 months, then provide the answer as 3 years. 
- For text/textarea fields: provide a professional, concise, and truthful answer. If resume does not provide an explicit answer, infer a suitable response that represents the candidate positively while aligning with company interests.
- For select/multi-select/radio/checkbox: choose the option(s) that best align with both the candidate’s skills/experience and the company’s perspective.
- Return **only the text or selected option(s)**, no extra commentary.
"""
        try:
            answer = ask_gemini(prompt)
        except Exception as e:
            print(f"❌ Error generating answer for field '{question}': {e}")
            answer = ""
        
        field_copy = field.copy()
        field_copy["generated_answer"] = answer.strip()
        answers.append(field_copy)
        time.sleep(0.5)  # rate-limit Gemini calls lightly
    return answers


# -----------------------------
# Save generated answers CSV
# -----------------------------
def save_answers_to_csv(fields_with_answers, filename="easy_apply_answers.csv"):
    file_exists = os.path.exists(filename)
    with open(filename, "a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["field_name", "field_type", "value", "options", "generated_answer"])
        if not file_exists:
            writer.writeheader()
        for row in fields_with_answers:
            row_copy = row.copy()
            row_copy.pop("element", None)
            row_copy["options"] = "; ".join(row_copy.get("options", [])) if row_copy.get("options") else ""
            writer.writerow(row_copy)
    print(f"✅ Answers appended to {filename}")


In [ ]:
import os
import time
import csv
import pickle
import json
import requests
import tempfile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

# -----------------------------
# Configuration
# -----------------------------
LINKEDIN_EMAIL = os.getenv("LINKEDIN_EMAIL")
LINKEDIN_PASSWORD = os.getenv("LINKEDIN_PASSWORD")
COOKIE_FILE = "linkedin_cookies.pkl"
RESUME_DRIVE_URL = "https://drive.google.com/file/d/1QgFWJDJS84TmvyRJeapjRUEtcEn_6QL9/view?usp=sharing"
CSV_PATH = "csv/linkedin_jobs.csv"

# -----------------------------
# Launch Chrome
# -----------------------------
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 10)

# -----------------------------
# Login functions
# -----------------------------
def try_login_with_cookies():
    if os.path.exists(COOKIE_FILE):
        driver.get("https://www.linkedin.com")
        with open(COOKIE_FILE, "rb") as f:
            cookies = pickle.load(f)
            for cookie in cookies:
                driver.add_cookie(cookie)
        driver.refresh()
        time.sleep(3)
        try:
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "img.global-nav__me-photo")))
            print("✅ Logged in using cookies")
            return True
        except:
            print("❌ Cookies expired or invalid")
            return False
    return False

def login_with_credentials():
    driver.get("https://www.linkedin.com/login")
    username_input = wait.until(EC.presence_of_element_located((By.ID, "username")))
    username_input.send_keys(LINKEDIN_EMAIL)
    password_input = driver.find_element(By.ID, "password")
    password_input.send_keys(LINKEDIN_PASSWORD)
    password_input.send_keys(Keys.RETURN)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "img.global-nav__me-photo")))
    with open(COOKIE_FILE, "wb") as f:
        pickle.dump(driver.get_cookies(), f)
    print("✅ Logged in and cookies saved")

# -----------------------------
# Easy Apply click & fallback
# -----------------------------
def click_easy_apply():
    try:
        easy_apply_button = driver.find_element(By.CSS_SELECTOR, "button.jobs-apply-button")
        if "applied" in easy_apply_button.text.strip().lower():
            print("ℹ️ Already applied. Skipping this job.")
            return "Already applied"
        driver.execute_script("arguments[0].scrollIntoView(true);", easy_apply_button)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", easy_apply_button)
        print("✅ Easy Apply clicked!")
        return "Clicked Easy Apply"
    except:
        print("ℹ️ Easy Apply button not found or job not available. Skipping.")
        return "Already applied"

# -----------------------------
# Resume download
# -----------------------------
def download_resume_from_drive(drive_url):
    file_id = drive_url.split("/d/")[1].split("/")[0]
    download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
    resp = requests.get(download_url, stream=True)
    if resp.status_code != 200:
        raise Exception(f"Failed to download resume, status code {resp.status_code}")
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf")
    for chunk in resp.iter_content(chunk_size=8192):
        if chunk:
            temp_file.write(chunk)
    temp_file.close()
    return temp_file.name

# -----------------------------
# Extract Easy Apply fields
# -----------------------------
def extract_easy_apply_fields():
    fields = []
    try:
        form_container = wait.until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "div.jobs-easy-apply-modal"))
        )

        # INPUTS
        for inp in form_container.find_elements(By.TAG_NAME, "input"):
            try:
                label_el = inp.find_element(By.XPATH, "ancestor::div[@data-test-single-line-text-form-component]//label")
                question_text = label_el.text.strip() if label_el else inp.get_attribute("aria-label") or inp.get_attribute("id")
                field_type = inp.get_attribute("type")
                value = inp.get_attribute("value") if field_type not in ["checkbox", "radio", "file"] else inp.is_selected()
                options = []

                if field_type in ["radio", "checkbox"]:
                    name_attr = inp.get_attribute("name")
                    group = form_container.find_elements(By.NAME, name_attr)
                    options = [el.get_attribute("aria-label") for el in group if el.get_attribute("aria-label")]

                if field_type == "file":
                    fields.append({
                        "field_name": question_text,
                        "field_type": "file",
                        "value": "",
                        "options": [],
                        "element": inp
                    })
                    continue

                fields.append({
                    "field_name": question_text,
                    "field_type": field_type,
                    "value": value,
                    "options": options,
                    "element": inp
                })
            except:
                continue

        # TEXTAREAS
        for ta in form_container.find_elements(By.TAG_NAME, "textarea"):
            try:
                label_el = ta.find_element(By.XPATH, "ancestor::div[@data-test-single-line-text-form-component]//label")
                question_text = label_el.text.strip() if label_el else ta.get_attribute("aria-label") or ta.get_attribute("id")
                fields.append({
                    "field_name": question_text,
                    "field_type": "textarea",
                    "value": ta.get_attribute("value"),
                    "options": [],
                    "element": ta
                })
            except:
                continue

        # SELECT / DROPDOWNS
        for sel in form_container.find_elements(By.TAG_NAME, "select"):
            try:
                label_el = sel.find_element(By.XPATH, "ancestor::div[@data-test-form-element]//label")
                question_text = label_el.text.strip() if label_el else sel.get_attribute("aria-label") or sel.get_attribute("id")
                options = [opt.text for opt in sel.find_elements(By.TAG_NAME, "option")]
                fields.append({
                    "field_name": question_text,
                    "field_type": "select",
                    "value": sel.get_attribute("value"),
                    "options": options,
                    "element": sel
                })
            except:
                continue

        # MULTI-SELECT / PILL-TYPE
        for ms in form_container.find_elements(By.CSS_SELECTOR, "div.artdeco-multiselect"):
            try:
                label_el = ms.find_element(By.XPATH, "ancestor::div[@data-test-form-element]//label")
                question_text = label_el.text.strip() if label_el else ms.get_attribute("aria-label") or ms.get_attribute("id")
                selected_items = [pill.text for pill in ms.find_elements(By.CSS_SELECTOR, "li span.artdeco-pill__text")]
                options_items = [li.text for li in ms.find_elements(By.TAG_NAME, "li")]
                fields.append({
                    "field_name": question_text,
                    "field_type": "multi-select",
                    "value": "; ".join(selected_items),
                    "options": options_items,
                    "element": ms
                })
            except:
                continue

    except:
        pass

    return fields

# -----------------------------
# Fill fields
# -----------------------------
def fill_easy_apply_fields(fields):
    for field in fields:
        try:
            el = field.get("element")
            if not el:
                continue
            generated = field.get("generated_answer", "").strip()
            f_type = field["field_type"]

            if f_type in ["text", "textarea"] or el.get_attribute("contenteditable") == "true":
                if generated:
                    driver.execute_script("""
                        arguments[0].focus();
                        arguments[0].value = arguments[1];
                        arguments[0].dispatchEvent(new Event('input', { bubbles: true }));
                        arguments[0].dispatchEvent(new Event('change', { bubbles: true }));
                    """, el, generated)
                    time.sleep(0.2)

            elif f_type == "select" and generated:
                try:
                    Select(el).select_by_visible_text(generated)
                except:
                    el.click()
                    time.sleep(0.3)
                    option = el.find_element(By.XPATH, f".//li[normalize-space(text())='{generated}']")
                    option.click()
                    time.sleep(0.2)

            elif f_type == "radio" and el.get_attribute("aria-label") == generated and not el.is_selected():
                el.click()

            elif f_type == "checkbox":
                should_check = el.get_attribute("aria-label") in generated.split(";")
                if el.is_selected() != should_check:
                    el.click()

            elif f_type == "multi-select":
                selected_items = field.get("value", "").split(";")
                to_select = generated.split(";")
                el.click()
                time.sleep(0.2)
                for li in el.find_elements(By.TAG_NAME, "li"):
                    text = li.text.strip()
                    if text in to_select and text not in selected_items:
                        li.click()
                        time.sleep(0.1)
                for li in el.find_elements(By.CSS_SELECTOR, "li span.artdeco-pill__text"):
                    text = li.text.strip()
                    if text not in to_select:
                        try:
                            remove_btn = li.find_element(By.XPATH, "..//button")
                            remove_btn.click()
                            time.sleep(0.1)
                        except:
                            continue

            elif f_type == "file":
                try:
                    local_file = download_resume_from_drive(RESUME_DRIVE_URL)
                    el.send_keys(local_file)
                    print(f"✅ Uploaded resume: {local_file}")
                    time.sleep(2)
                    if os.path.exists(local_file):
                        os.remove(local_file)
                        print(f"🗑️ Deleted temp resume file: {local_file}")
                except:
                    pass
        except:
            continue

# -----------------------------
# Gemini LLM answer generator placeholder
# -----------------------------
def generate_field_answers(fields, resume_json):
    # Replace this function with your LLM API call
    for field in fields:
        field["generated_answer"] = "Sample answer"  # placeholder
    return fields

# -----------------------------
# Main automation loop
# -----------------------------
if not try_login_with_cookies():
    login_with_credentials()

with open("resumes/Yeswanth_Yerra_CV_structured.json", "r") as f:
    resume_data = json.load(f)

# Read CSV
with open(CSV_PATH, "r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    jobs = [row for row in reader if row["apply_type"].lower() == "easy apply"]

for idx, job in enumerate(jobs, 1):
    print(f"\n🎯 Processing job {idx}/{len(jobs)}: {job['title']} at {job['company']} ({job['location']})")
    driver.get(job["apply_link"])
    time.sleep(3)

    apply_status = click_easy_apply()
    if apply_status == "Already applied":
        continue

    step = 1
    while True:
        print(f"➡️ Step {step}...")
        fields = extract_easy_apply_fields()
        if not fields:
            print("ℹ️ No fields detected, likely final review step.")
            break

        fields_with_answers = generate_field_answers(fields, resume_data)
        fill_easy_apply_fields(fields_with_answers)

        # Next/Review/Continue
        try:
            next_button = wait.until(
                EC.presence_of_element_located(
                    (By.XPATH, "//button[contains(., 'Next') or contains(., 'Review') or contains(., 'Continue')]")
                )
            )
            if "disabled" in next_button.get_attribute("class"):
                break
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            time.sleep(0.5)
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(2)
            step += 1
        except:
            break

    # Final Submit
    try:
        submit_button = wait.until(
            EC.presence_of_element_located(
                (By.XPATH, "//button[contains(., 'Submit') or contains(., 'Done')]")
            )
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", submit_button)
        time.sleep(0.5)
        driver.execute_script("arguments[0].click();", submit_button)
        print("✅ Application submitted successfully!")
        time.sleep(2)
    except:
        print("⚠️ Could not find Submit button, skipped.")

driver.quit()
print("🎉 All Easy Apply jobs processed!")


✅ Logged in using cookies

🎯 Processing job 1/17: Data Scientist at AB InBev GCC India (Bengaluru, Karnataka, India)
ℹ️ Easy Apply button not found or job not available. Skipping.

🎯 Processing job 2/17: Associate - Data Scientist-Data Science-Data Scientist at EXL (Bengaluru, Karnataka, India)
ℹ️ Easy Apply button not found or job not available. Skipping.

🎯 Processing job 3/17: Senior Data Scientist at Tata Communications Transformation Services (TCTS) (Pune, Maharashtra, India)
ℹ️ Easy Apply button not found or job not available. Skipping.

🎯 Processing job 4/17: Machine Learning Engineer at Persistent Systems (Pune, Maharashtra, India)
ℹ️ Easy Apply button not found or job not available. Skipping.

🎯 Processing job 5/17: AI/ML Engineer at CG-VAK Software & Exports Ltd. (Mumbai Metropolitan Region)
✅ Easy Apply clicked!
➡️ Step 1...
➡️ Step 2...
ℹ️ No fields detected, likely final review step.
⚠️ Could not find Submit button, skipped.

🎯 Processing job 6/17: AI/ML Engineer at Impetu

InvalidSessionIdException: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
#0 0x604fb76bcfea <unknown>
#1 0x604fb713ab4e <unknown>
#2 0x604fb717e459 <unknown>
#3 0x604fb71b2d16 <unknown>
#4 0x604fb71ad496 <unknown>
#5 0x604fb71ac5d2 <unknown>
#6 0x604fb7102b65 <unknown>
#7 0x604fb76809e8 <unknown>
#8 0x604fb768484f <unknown>
#9 0x604fb7667ec9 <unknown>
#10 0x604fb76853f5 <unknown>
#11 0x604fb764d74f <unknown>
#12 0x604fb7100a00 <unknown>
#13 0x7dc9f0429d90 <unknown>
